# Extraction des données complètes pour le Rhones

In [7]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import json
import pandas as pd

# URL de base de l'API
base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"

In [8]:
# Chargement du ficher des adresses du rhones.
df69 = pd.read_csv("data/adresses-69.csv", sep=";")

C:\Users\axel-\AppData\Local\Temp\ipykernel_74572\332036706.py:2: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df69 = pd.read_csv("data/adresses-69.csv", sep=";")


In [33]:
# Extraction des codes postaux.
code_postaux = df69["code_postal"].unique()

# Clasic method

In [43]:
# Récupération de 500 lignes max par code postal.

def query_api(params: dict, next_url: str | None = None) -> list[dict]:
    """Récupère toutes les pages d'une requête API en boucle."""
    all_results = []

    # Compteur de boucle.
    count = 0

    while True:

        if count == 5:
            break

        if next_url:
            response = requests.get(url=next_url)
        else:
            response = requests.get(url=base_url, params=params)

        # Si problème.
        if response.status_code != 200:
            print("Erreur de connexion avec l'API!")
            return None

        content = response.json()

        print(f"Récupération de {params["size"]} éléments sur {content["total"] - (count * params["size"])}")

        all_results.extend(content["results"])

        next_url = content.get("next")
        count += 1

        # Si plus de pages détectées: casser la boucle.
        if not next_url:
            break 

    return all_results

df_list = list()

for i, code in enumerate(code_postaux):

    params = {
            "size": 100,   
            "page": 1,
            "qs": f"code_postal_ban:{code}"
        }

    print("----Récupération des données du code postal: " + str(code) + f" ({i+1}/{len(code_postaux)})" + "----")
    df_list.extend(query_api(params=params))

# Concaténation de la liste de dictionnaire en dataframe.
df = pd.DataFrame(df_list)

df.to_csv("existants_69.csv")


----Récupération des données du code postal: 69790 (1/91)----
Récupération de 100 éléments sur 197
Récupération de 100 éléments sur 97
----Récupération des données du code postal: 69170 (2/91)----
Récupération de 100 éléments sur 2893
Récupération de 100 éléments sur 2793
Récupération de 100 éléments sur 2693
Récupération de 100 éléments sur 2593
Récupération de 100 éléments sur 2493
----Récupération des données du code postal: 69250 (3/91)----
Récupération de 100 éléments sur 3167
Récupération de 100 éléments sur 3067
Récupération de 100 éléments sur 2967
Récupération de 100 éléments sur 2867
Récupération de 100 éléments sur 2767
----Récupération des données du code postal: 69380 (4/91)----
Récupération de 100 éléments sur 3216
Récupération de 100 éléments sur 3116
Récupération de 100 éléments sur 3016
Récupération de 100 éléments sur 2916
Récupération de 100 éléments sur 2816
----Récupération des données du code postal: 69009 (5/91)----
Récupération de 100 éléments sur 21052
Récupéra

## Multitreading code

In [26]:
# URL de base

def query_api(params: dict, next_url: str | None = None) -> pd.DataFrame | None:
    """Récupère toutes les pages d'une requête API en boucle."""

    all_results = []

    while True:
        if next_url:
            response = requests.get(url=next_url)
        else:
            response = requests.get(url=base_url, params=params)

        if response.status_code != 200:
            print("Erreur de connexion avec l'API!")
            return None

        content = response.json()

        # Génère une liste de dictionnaires de chaque résultat.
        all_results.extend(content["results"])

        print(f"Récupération de {params["qs"]} (total: {content["total"]})")

        # Récupère l'URL de la page suivante
        next_url = content.get("next")
        
        if not next_url:
            break

    return pd.DataFrame(all_results)  # Concatène la liste de dictionnaire en dataframe.


def fetch_for_code(code):
    """Exécute query_api pour un code postal donné."""
    print("Récupération des données de " + str(code))

    params = {
        "size": 1000,   
        "page": 1,
        "qs": f"code_postal_ban:{code}"
    }
    return query_api(params=params)


# Liste des DataFrames pour tous les codes postaux
with ThreadPoolExecutor(max_workers=5) as executor: 
    df_list = list(executor.map(fetch_for_code, code_postaux))

# Concaténation finale de toutes les données
df_final = pd.concat(df_list, ignore_index=True)

Récupération des données de 69790
Récupération des données de 69170
Récupération des données de 69250
Récupération des données de 69380
Récupération des données de 69009
Récupération de code_postal_ban:69790 (total: 197)
Récupération des données de 69008
Récupération de code_postal_ban:69380 (total: 3216)
Récupération des données de 69006
Récupération de code_postal_ban:69006 (total: 15390)
Récupération de code_postal_ban:69250 (total: 3167)
Récupération de code_postal_ban:69250 (total: 3167)
Récupération de code_postal_ban:69250 (total: 3167)
Récupération de code_postal_ban:69250 (total: 3167)
Récupération de code_postal_ban:69006 (total: 15390)
Récupération de code_postal_ban:69006 (total: 15390)
Récupération de code_postal_ban:69006 (total: 15390)
Récupération de code_postal_ban:69006 (total: 15390)
Récupération de code_postal_ban:69006 (total: 15390)
Récupération de code_postal_ban:69006 (total: 15390)
Récupération de code_postal_ban:69006 (total: 15390)
Récupération de code_postal

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))